In [1]:
import os
import torch
import torch_tensorrt

/home/andrew/.mambaforge/lib/python3.10/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: '/home/andrew/.mambaforge/lib/python3.10/site-packages/torchvision/image.so: undefined symbol: _ZN3c104cuda20CUDACachingAllocator9allocatorE'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


In [2]:
ssd_checkpoint_file = "/home/andrew/Development/wildfire/WIT-Asset-Detection/ssd_logs/2023_03_01__13_42_11/andrew_desktop_HIT+WIT_550_checkpoint_ssd300_state_dict.pt"
output_file_name = (
    os.path.splitext(os.path.basename(ssd_checkpoint_file))[0] + ".torchscript"
)
print(f"{output_file_name=}")

output_file_name='andrew_desktop_HIT+WIT_550_checkpoint_ssd300_state_dict.torchscript'


In [3]:
checkpoint = torch.load(ssd_checkpoint_file)
start_epoch = checkpoint["epoch"] + 1
print("\nLoaded checkpoint from epoch %d.\n" % start_epoch)


from ssd_model import SSD300
from ssd_utils import label_map

n_classes = len(label_map)
model = SSD300(n_classes=n_classes)

model.load_state_dict(checkpoint["model"])
model.eval()


Loaded checkpoint from epoch 551.


Loaded base model.

tensor([[0.0132, 0.0132, 0.1000, 0.1000],
        [0.0132, 0.0132, 0.1414, 0.1414],
        [0.0132, 0.0132, 0.1414, 0.0707],
        ...,
        [0.5000, 0.5000, 1.0000, 1.0000],
        [0.5000, 0.5000, 1.0000, 0.6364],
        [0.5000, 0.5000, 0.6364, 1.0000]])


SSD300(
  (base): VGGBase(
    (conv1_1): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (conv1_2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (pool1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (conv2_1): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (conv2_2): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (pool2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (conv3_1): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (conv3_2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (conv3_3): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (pool3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=True)
    (conv4_1): Conv2d(256, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (conv4_2): Conv2d(512, 512, kernel_size=(3, 3), 

In [4]:
from dataset import HITUAVDatasetTrain, HITUAVDatasetVal, WITUAVDataset, CombinedDataset
from torch.utils.data import Dataset, DataLoader
import albumentations as A

image_transform = A.Compose(
    [
        # Randomly rotate the image and bounding boxes
        A.RandomRotate90(),
        # Randomly flip the image and bounding boxes horizontally
        A.HorizontalFlip(p=0.5),
        # Randomly flip the image and bounding boxes vertically
        A.VerticalFlip(p=0.5),
        # Apply a color jitter to the image
        A.ColorJitter(p=0.5),
        # Apply CLAHE to the image
        A.CLAHE(clip_limit=(1, 4), p=0.5),
        # Add random noise to the image
        A.GaussNoise(var_limit=(10.0, 50.0), p=0.5),
        # Add a grid of lines to the image
        A.GridDistortion(p=0.5),
        # Randomly change brightness, contrast and saturation
        A.RandomBrightnessContrast(p=0.5),
        A.RandomGamma(p=0.5),
        A.HueSaturationValue(p=0.5),
        # Add a black rectangle to the image
        A.RandomShadow(
            p=0.5, num_shadows_lower=1, num_shadows_upper=3, shadow_dimension=3
        ),
        A.RandomResizedCrop(
            height=300,
            width=300,
            scale=(0.8, 1.0),
            ratio=(0.75, 1.3333333333333333),
            p=1.0,
        ),
        A.Perspective(),
    ],
    # HIT dataset is in unnormalized YOLO format, which is x_center, y_center, width, height.
    # however for SSD it transforms it to pascal_voc format
    bbox_params=A.BboxParams(format="pascal_voc", label_fields=["labels"]),
)

train_dataset = WITUAVDataset(
    root="./WIT-UAV-Dataset_split/train/",
    sensor="both",
    image_transform=image_transform,
)
train_loader = torch.utils.data.DataLoader(
    train_dataset,
    batch_size=1,
    shuffle=False,
    collate_fn=train_dataset.collate_fn,
    num_workers=4,
    pin_memory=True,
)  # note that we're passing the collate function here

In [5]:
inputs = [
    torch_tensorrt.Input(
        min_shape=[1, 3, 300, 300],
        opt_shape=[1, 3, 300, 300],
        max_shape=[1, 3, 300, 300],
        dtype=torch.half,
    )
]
enabled_precisions = {torch.half}  # Run with fp16

trt_ts_module = torch_tensorrt.compile(
    model.half().cuda(), inputs=inputs, enabled_precisions=enabled_precisions
)

input_data = torch.random.randn
input_data = next(iter(train_loader))[0]
input_data = input_data.to("cuda").half()
result = trt_ts_module(input_data)
torch.jit.save(
    trt_ts_module, "andrew_desktop_HIT+WIT_550_checkpoint_ssd300_trt_half.torchscript"
)